In [1]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload complete

In [2]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import dask
import cftime

from lmrecon.io import open_mfdataset
from lmrecon.plotting import plot_field
from lmrecon.util import get_data_path, unstack_state
from lmrecon.spaces import PhysicalSpaceForecastSpaceMapper
from lmrecon.logger import logging_disabled

In [3]:
dataset = "2024-06-03T01-20-30"
data = open_mfdataset(get_data_path() / "reduced_space" / dataset / "seasonal_anomalies")["data"]
data

In [4]:
mapper = PhysicalSpaceForecastSpaceMapper.load(
    get_data_path() / "reduced_space" / dataset / "mapper.pkl"
)

In [5]:
field = "tas"

# Physical state reconstruction
# with logging_disabled():
#     state = unstack_state(mapper.backward(data.isel(time=0)))[field]

# EOF
state = (
    xr.DataArray(
        mapper.nan_masks[field].backward(
            mapper.eofs_individual[field].get_component(0)[:, np.newaxis]
        ),
        coords=dict(state=mapper.state_coords.sel(field=field), time=[0]),
    )
    .unstack("state")
    .squeeze()
)

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.EqualEarth(central_longitude=198)))
plot_field(ax, state, cmap="RdBu_r", same_limits=True)
ax.coastlines()